# Unified Model: Limit Opinions with Bots

James Yu, 7 August 2022

In [1]:
import matplotlib.pyplot as plt
import numpy as np

# Case 1: generic network, no patterns, but all $x_0^i = 0$

In [2]:
A_1 = np.array([
    [0.3, 0.4, 0.3],
    [0.6, 0.2, 0.2],
    [0.5, 0.1, 0.4]
])
delta = 0.9
c = 1.0
x_0 = np.array([[0.0, 0.0, 0.0]], ndmin = 2).T
z = np.array([[5.0, 5.0, 5.0]], ndmin = 2).T # the bot "messages"

Linear State Space Model reparametrization:

In [3]:
n = 3
Q = np.block([[np.identity(n), np.zeros((n, n))], [np.zeros((n, n)), np.zeros((n, n))]])
Q

array([[1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [4]:
Ae = np.block([[A_1, np.identity(n)], [np.zeros((n, n)), np.identity(n)]])
Ae

array([[0.3, 0.4, 0.3, 1. , 0. , 0. ],
       [0.6, 0.2, 0.2, 0. , 1. , 0. ],
       [0.5, 0.1, 0.4, 0. , 0. , 1. ],
       [0. , 0. , 0. , 1. , 0. , 0. ],
       [0. , 0. , 0. , 0. , 1. , 0. ],
       [0. , 0. , 0. , 0. , 0. , 1. ]])

In [5]:
B = np.block([[np.identity(n)], [np.zeros((n, n))]])
B

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 0.],
       [0., 0., 0.],
       [0., 0., 0.]])

In [6]:
chi_0 = np.block([[x_0], [z]])
chi_0

array([[0.],
       [0.],
       [0.],
       [5.],
       [5.],
       [5.]])

Then:

$$\chi_{t+1} = A^e \chi_t + B r_t$$

In [7]:
def infinite_solution_general(A, delta, c, x_0, z):
    # given that this model uses the general setup (without projections), 
    # we can adapt the old code:
    n = len(x_0)
    Q = np.block([[np.identity(n), np.zeros((n, n))], [np.zeros((n, n)), np.zeros((n, n))]])
    Ae = np.block([[A, np.identity(n)], [np.zeros((n, n)), np.identity(n)]])
    B = np.block([[np.identity(n)], [np.zeros((n, n))]])
    K_t = Q
    K_sequence = [K_t]
    
    while True: # generate solution matrices
        K_t_new = Q + delta * (Ae.T @ (K_t - ((delta * K_t) @ B @ np.linalg.inv((delta * (B.T @ K_t @ B)) + (c * np.identity(n))) @ B.T @ K_t)) @ Ae)
        K_sequence.insert(0, K_t_new)
        if np.allclose(K_t, K_t_new, 
                       rtol = np.finfo(K_t.dtype).eps, 
                       atol = np.finfo(K_t.dtype).eps):
            break
        K_t = K_t_new

    chi_t_iterator = np.block([[x_0], [z]])
    chi_ts = [chi_t_iterator]
    r_ts = []
    K_ss = K_sequence[0]
    payoff = 0
    payoffs = []
    i = 0
    while True:
        L_ss = -delta * (np.linalg.inv((delta * (B.T @ K_ss @ B)) + (c * np.identity(n))) @ B.T @ K_ss @ Ae)
        r_t = L_ss @ chi_t_iterator
        r_ts.append(r_t)
        
        payoff += (-1 * delta**i * (chi_t_iterator.T @ Q @ chi_t_iterator)).item() + (-1 * delta**i * c * (r_t.T @ r_t)).item()
        payoffs.append(payoff)
        chi_t_iterator_new = (Ae @ chi_t_iterator) + (B @ r_t)
        chi_ts.append(chi_t_iterator_new)
        if np.allclose(chi_t_iterator, chi_t_iterator_new,
                       rtol = 10**-15, 
                       atol = 10**-15): # reduced precision due to apparent error buildup
            break
        chi_t_iterator = chi_t_iterator_new
        i += 1
        
    return chi_ts, r_ts, payoffs

In [8]:
chi_ts, r_ts, payoffs = infinite_solution_general(A_1, delta, c, x_0, z)

In [9]:
chi_ts

[array([[0.],
        [0.],
        [0.],
        [5.],
        [5.],
        [5.]]),
 array([[-0.40018038],
        [ 0.96009637],
        [ 0.58831726],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[-0.21022466],
        [ 0.9774515 ],
        [ 0.63145493],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[-0.18494416],
        [ 1.03545976],
        [ 0.68124527],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[-0.16731575],
        [ 1.05147317],
        [ 0.69723519],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[-0.16121547],
        [ 1.05904934],
        [ 0.70438386],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[-0.15858271],
        [ 1.06197454],
        [ 0.70720082],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[-0.15753423],
        [ 1.06318603],
        [ 0.7083594 ],
     

In [10]:
r_ts

[array([[-5.40018038],
        [-4.03990363],
        [-4.41168274]]),
 array([[-5.65070427],
        [-4.092123  ],
        [-4.49979142]]),
 array([[-5.70229384],
        [-4.16018673],
        [-4.56396953]]),
 array([[-5.73038998],
        [-4.18090134],
        [-4.58633682]]),
 array([[-5.74078057],
        [-4.19030288],
        [-4.59599966]]),
 array([[-5.74515297],
        [-4.19398282],
        [-4.59984992]]),
 array([[-5.74690948],
        [-4.19549941],
        [-4.60142703]]),
 array([[-5.74762673],
        [-4.19611312],
        [-4.60206664]]),
 array([[-5.74791793],
        [-4.19636307],
        [-4.60232694]]),
 array([[-5.74803639],
        [-4.19646464],
        [-4.60243275]]),
 array([[-5.74808455],
        [-4.19650595],
        [-4.60247577]]),
 array([[-5.74810413],
        [-4.19652274],
        [-4.60249327]]),
 array([[-5.74811209],
        [-4.19652957],
        [-4.60250038]]),
 array([[-5.74811533],
        [-4.19653235],
        [-4.60250327]]),
 array

In [11]:
payoffs

[-64.94571404703164,
 -128.26260295539223,
 -186.62429183201385,
 -239.7846245688658,
 -287.8492901397677,
 -331.19044504041403,
 -370.227648819729,
 -405.37221164819886,
 -437.0063696704483,
 -465.478595221055,
 -491.1041409839496,
 -514.1673308039551,
 -534.9242743292386,
 -553.6055501013942,
 -570.4187080301026,
 -585.5505537279163,
 -599.1692161594167,
 -611.4260128247578,
 -622.4571299981145,
 -632.3851355180103,
 -641.3203405092908,
 -649.3620250099967,
 -656.5995410637622,
 -663.1133055132966,
 -668.9756935182967,
 -674.2518427229502,
 -679.0003770071944,
 -683.2740578630347,
 -687.1203706332985,
 -690.5820521265388,
 -693.697565470456,
 -696.5015274799819,
 -699.0250932885552,
 -701.2963025162713,
 -703.3403908212158,
 -705.1800702956659,
 -706.835781822671,
 -708.3259221969756,
 -709.6670485338498]

In [12]:
len(chi_ts)

40

# Case 2: perfectly uniform network

In [13]:
A_2 = np.array([
    [1/3, 1/3, 1/3],
    [1/3, 1/3, 1/3],
    [1/3, 1/3, 1/3]
])

In [14]:
chi_ts, r_ts, payoffs = infinite_solution_general(A_2, delta, c, x_0, z)
chi_ts[-1]

array([[0.55555556],
       [0.55555556],
       [0.55555556],
       [5.        ],
       [5.        ],
       [5.        ]])

In [15]:
r_ts[-1]

array([[-5.],
       [-5.],
       [-5.]])

In [16]:
len(chi_ts)

40

# Case 3: isolated network

In [17]:
A_3 = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]
])

chi_ts, r_ts, payoffs = infinite_solution_general(A_3, delta, c, x_0, z)
chi_ts[-1]

array([[0.55555556],
       [0.55555556],
       [0.55555556],
       [5.        ],
       [5.        ],
       [5.        ]])

In [18]:
r_ts[-1]

array([[-5.],
       [-5.],
       [-5.]])

In [19]:
len(chi_ts)

40

# Case 4: periodic network

In [20]:
A_4 = np.array([
    [0, 1, 0],
    [1, 0, 0],
    [0, 0, 1] # last agent is independent
])

x_0_case4 = np.array([[-10.0, 5.0, -3.0]], ndmin = 2).T
chi_ts, r_ts, payoffs = infinite_solution_general(A_4, delta, c, x_0_case4, z)
chi_ts[-1]

array([[0.55555556],
       [0.55555556],
       [0.55555556],
       [5.        ],
       [5.        ],
       [5.        ]])

In [21]:
A_4 @ x_0_case4

array([[  5.],
       [-10.],
       [ -3.]])

In [22]:
A_4 @ A_4 @ x_0_case4

array([[-10.],
       [  5.],
       [ -3.]])

In [23]:
A_4 @ A_4 @ A_4 @ x_0_case4 # etc

array([[  5.],
       [-10.],
       [ -3.]])

In [24]:
chi_ts

[array([[-10.],
        [  5.],
        [ -3.],
        [  5.],
        [  5.],
        [  5.]]),
 array([[ 2.384874  ],
        [-3.78907576],
        [-0.9078992 ],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[-1.23268014],
        [ 1.3084969 ],
        [-0.04679752],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[ 0.86546369],
        [-0.18047627],
        [ 0.30762905],
        [ 5.        ],
        [ 5.        ],
        [ 5.        ]]),
 array([[0.25260732],
        [0.68311271],
        [0.45350983],
        [5.        ],
        [5.        ],
        [5.        ]]),
 array([[0.60805765],
        [0.43086308],
        [0.51355388],
        [5.        ],
        [5.        ],
        [5.        ]]),
 array([[0.50423255],
        [0.57716524],
        [0.53826781],
        [5.        ],
        [5.        ],
        [5.        ]]),
 array([[0.56445003],
        [0.53443118],
        [0.54843998],
        [5.   

In [25]:
r_ts

[array([[-7.615126  ],
        [ 1.21092424],
        [-2.9078992 ]]),
 array([[-2.44360438],
        [-6.0763771 ],
        [-4.13889832]]),
 array([[-5.44303321],
        [-3.94779613],
        [-4.64557343]]),
 array([[-4.56691641],
        [-5.18235099],
        [-4.85411921]]),
 array([[-5.07505505],
        [-4.82174424],
        [-4.93995596]]),
 array([[-4.92663053],
        [-5.03089241],
        [-4.97528607]]),
 array([[-5.01271521],
        [-4.96980137],
        [-4.98982783]]),
 array([[-4.98757035],
        [-5.00523354],
        [-4.99581317]]),
 array([[-5.00215411],
        [-4.994884  ],
        [-4.99827671]]),
 array([[-4.99789427],
        [-5.00088662],
        [-4.9992907 ]]),
 array([[-5.00036493],
        [-4.99913329],
        [-4.99970806]]),
 array([[-4.99964326],
        [-5.0001502 ],
        [-4.99987984]]),
 array([[-5.00006182],
        [-4.99985317],
        [-4.99995054]]),
 array([[-4.99993956],
        [-5.00002545],
        [-4.99997964]]),
 array